In [153]:
from tkinter import *
from functools import partial
from Census21_CACD_Wrapper import APIWrapper
FONT = ("arial",10)

In [187]:
dimension_drop_1 = None
dimension_drop_2 = None
regional_drop = None

def configure_api():
    api = APIWrapper()
    return api
    
# Create a selection w/ button
def drop_config_button(text, selection, row, column, command):
    # Label config.
    label = Label(text=text, font=FONT)
    label.grid(row=row, column=column)
    # Drop down config.
    variable = StringVar(window)
    drop_list = OptionMenu(window, variable, *selection)
    variable.set(text)
    drop_list.grid(row=row+1, column=column)
    # Button config
    residence_select = Button(text="Select", command=command)
    residence_select.grid(row=row+2, column=column)
    return drop_list


# Create a selection w/o button
def drop_config(text, selection, row, column):
    # Label config.
    label = Label(text=text, font=FONT)
    label.grid(row=row, column=column)
    # Drop down config.
    variable = StringVar(window)
    drop_list = OptionMenu(window, variable, *selection)
    variable.set(text)
    drop_list.grid(row=row+1, column=column)
    return drop_list


# Retrieves the selection of residence drop list & return abbreviation
def retrieve_res_input():
    output = residence_drop.cget('text')
    if output == 'Usual Residence':
        residence_code = 'UR'
    elif output == 'Household':
        residence_code = 'HH'
    else:
        residence_code = 'HRP'
        
     # Create a list of possible dimensions for this res code
    dimension_list = list_dimensions(residence_code)
    create_dimension_selection(dimension_list, residence_code)



def execute_api_call(residence_code, dimension_drop_1, dimension_drop_2, regional_drop):
    dim_1 = dimension_drop_1.cget('text')
    dim_2 = dimension_drop_2.cget('text')
    region_dict = api.get_areas_by_pop_type(residence_code)
    region_value = regional_drop.cget('text')
    region = region_dict[region_value]
    data = api.query_api(residence_code, f'{dim_1},{dim_2}', region)
    with open(f'data/output/{residence_code}_{dim_1}_{dim_2}_{region}.csv', 'w') as file:
        file.write(data.to_string(index=False))
    print(data)

    
# Create remaining 3 drop down lists.
def create_dimension_selection(dimension_list, residence_code):
    global dimension_drop_1
    global dimension_drop_2
    global regional_drop
    # Retrieve list of dimensions 
    dimension_drop_1 = drop_config('Variable 1', dimension_list, 
                             row=2, column=2) 
    dimension_drop_2 = drop_config('Variable 2', dimension_list, 
                             row=2, column=3)
    area_codes = list_areas(residence_code)
    regional_drop = drop_config('Regional Divisions', 
                                            area_codes, row=2, column=4)
    but = Button( text = 'Save Data', command=partial(execute_api_call, residence_code, dimension_drop_1, dimension_drop_2, regional_drop))
    but.grid(column=4, row=4)
    
                                       
# Returns the list of demensions relating to that reFapisidence code (HH, HRP, UR) & add option for None
def list_dimensions(residence_code):
    dimension_list = list(api.get_dims_by_pop_type(residence_code).values())
    dimension_list.append('None')
    return dimension_list


# returns a list of areas relating to the selected residence code
def list_areas(residence_code):
    area_list = list(api.get_areas_by_pop_type(residence_code).keys())
    return area_list


def start_selections():
    residence_drop = drop_config_button('Residence Selection', ('Usual Residence', 'Household', 'Household Reference Person'), 
                             row=2, column=1, command=retrieve_res_input)
    return residence_drop
   
# Configure Window
window = Tk()
window.title('Census API')
window.configure(padx=50, pady=50)
api = configure_api()
residence_drop = start_selections()

# Keep window running
window.mainloop()